# 2016-00-03 07:55:37 

## Setup 

In [ ]:
import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy  
import scikits.bootstrap as bootstrap
from scipy import stats
pd.__version__

In [ ]:
%matplotlib inline

In [ ]:
import scipy  
from scipy import stats
import scikits.bootstrap as bootstrap  
scipy.__version__

### Data Specifics

In [ ]:
ls

In [ ]:
averages_header_row=['update',
                     'merit',
                     'gestation_time',
                     'fitness',
                     'repro_rate',
                     'size',
                     'copied_size',
                     'executed_size',
                     'abundance',
                     'prop_birthers',
                     'prop_breedtrue',
                     'genotype_depth',
                     'generation',
                     'neutral_metric',
                     'lineage_label',
                     'true_rep_rate'
                     ]

### Functions

In [ ]:
averages_header_row=['update',
                     'merit',
                     'gestation_time',
                     'fitness',
                     'repro_rate',
                     'size',
                     'copied_size',
                     'executed_size',
                     'abundance',
                     'prop_birthers',
                     'prop_breedtrue',
                     'genotype_depth',
                     'generation',
                     'neutral_metric',
                     'lineage_label',
                     'true_rep_rate'
                     ]
# read data into a dict collection of arrays of data frames.
# the dict is indexed by the treatment
# each item in the array under the dict is a data-frame that is
# a replicate of that treatment.
# treatment_pattern is "P_"+prob+"/hgtbonus"
# columns is the set of columns we want to keep.
def read_treatments(treatment_pattern, target_filename, columns=None, verbose=False, names=None):
    if verbose:
        print "READ TREATMENTS"
    data = {}
    for folder in glob.glob("raw_data/"+treatment_pattern):
        if verbose:
            print folder
        name = folder.split("/")[2]
        if verbose:
            print name
        data[name] = []

#        print folder + "/*/data*/"+target_filename
#        print glob.glob(folder + "/*/data*/"+target_filename)
        for datafile in glob.glob(folder + "/*/data*/"+target_filename):
            #print datafile
            bleh = pd.read_csv(datafile,
                               names=names, 
                               delimiter=" ", 
                               comment='#',
                               usecols=columns,
                               index_col=False)
            data[name].append(bleh)
    if verbose:
        print "DONE"
    return data

## tests
thing = read_treatments("P_1/hgtbonus", "average.dat", 
                        columns=['update', 'fitness'], 
                        names=averages_header_row, verbose=True)
print thing.keys()
print thing['hgtbonus']
thing = read_treatments("P_1/*", "average.dat", names=averages_header_row, verbose=True)
print thing.keys()
print thing['hgtbonus']


In [ ]:
# dataframes are an array of dataframes, one per replicate.
def group_treatment(data_frames, verbose=False):
    ## data_frames is a [] of dataframes that are all replicates of each other.
    #print data_frames
    if verbose:
        print "Incoming:"
        print len(data_frames)    
    aggregated_dataframe = (pd.concat(data_frames, axis=1, keys=range(len(data_frames)))
                            .swaplevel(0, 1, axis=1)
                            .sortlevel(axis=1)
                            .groupby(level=0, axis=1))
    if verbose:
        print "Out:"
        print aggregated_dataframe.head()

    return aggregated_dataframe


## Tests
treatments = read_treatments("P_1/*", "average.dat", 
                        columns=['update', 'fitness'], 
                        names=averages_header_row)
print treatments.keys()

aggregated_treatments = {}
for df_set_key in treatments.keys():
    print df_set_key
    aggregated_treatments[df_set_key] = group_treatment(treatments[df_set_key], verbose=True)
    #print aggregated_treatments[df_set_key].describe()


In [ ]:
def calc_stats(aggregated_dataframe, verbose=False):
    # calculate stats for data

    means = aggregated_dataframe.mean()
    stderrs = aggregated_dataframe.std().div(np.sqrt(len(aggregated_dataframe))).mul(2.0)
    
    if verbose:
        print "Means:"
        print means.head()
        print "STDErrs:"
        print stderrs.head()
    
    return (means, stderrs)

## Tests
treatments = read_treatments("P_1/hgtbonus", "average.dat", 
                        columns=['update', 'fitness'], 
                        names=averages_header_row)
print treatments.keys()

aggregated_treatments = {}
aggregated_means = {}
for df_set_key in treatments.keys():
    aggregated_treatments[df_set_key] = group_treatment(treatments[df_set_key], verbose=False)
    (means, stds) = calc_stats(aggregated_treatments[df_set_key], verbose=True)

   

In [ ]:
def bootstrap_error( data , verbose=False, debug=False):
    x = np.array(data)
    if debug:
        print len(x)
        #print x
    X = [] ## estimates
    mean = np.mean(x)
    for xx in xrange(1000): ## do this 1000 times
        X.append( np.mean( x[np.random.randint(len(x),size=len(x))] ) )
    if debug:
        print len(X)
        #print X
                
    ## re-sample means are not guaranteed to be quite right.
    ## Conf 0.95, loc=sample mean, scale = (np.std(X, ddof=1)/np.sqrt(len(X)))
    conf_int = stats.norm.interval(0.95, loc=np.mean(X), scale=stats.sem(X))
    if debug:
        print conf_int

    return conf_int

def calc_bootstrap_error(aggDF, verbose=False, debug=False):
    bsErrDF = pd.DataFrame()
    for key, subset in aggDF:
        if verbose:
            print "Key: " + key
            print subset.head()
            print len(subset)
        
        CIs = []
        for i in range(len(subset)):
            vals = subset.iloc[i].values
            if debug:
                print vals
            CIs.append(bootstrap_error(vals, verbose=verbose, debug=debug))
        
        if verbose:
            print CIs
            
        bsErrDF[key] = CIs
        #bleh = subset.apply(lambda x:bootstrap.ci(data=x.values))
        #print bleh

    if verbose:
        print "BOOTSTRAP VALS:"
        print bsErrDF.head()
        
    return bsErrDF


## Tests
treatments = read_treatments("P_1/hgtbonus", "average.dat", 
                        columns=['update', 'fitness'], 
                        names=averages_header_row)
print treatments.keys()

aggregated_treatments = {}
aggregated_means = {}
for df_set_key in treatments.keys():
    aggregated_treatments[df_set_key] = group_treatment(treatments[df_set_key], verbose=False)
    #(means, stds) = calc_stats(aggregated_treatments[df_set_key], verbose=True)
    bootstrap_err = calc_bootstrap_error(aggregated_treatments[df_set_key], verbose=True)

In [ ]:
def plot_column(column, meanDFs, stderrDFs):
    

    plt.figure(figsize=(20, 15))
    plt.title(column.replace("_", " ").title())
    plt.ylabel(column.replace("_", " ").title())
    plt.xlabel("update")

    for key in meanDFs.keys():

        plt.errorbar(x=meanDFs[key]["update"], 
                     y=meanDFs[key][column], 
                     yerr=stderrDFs[key][column], 
                     label=key)

    plt.legend(loc=2)
    
plot_column('fitness', means_t, bootstraps_t)

In [ ]:
def plot_sets(column, treatment, means, errors):
    
    plt.figure(figsize=(20, 15))
    plt.title(column.replace("_", " ").title())
    plt.ylabel(column.replace("_", " ").title())
    plt.xlabel("update")

    for treatmentset in means.keys():

        plt.errorbar(x=means[treatmentset][treatment]["update"], # mean
                     y=means[treatmentset][treatment][column], # mean
                     yerr=errors[treatmentset][treatment][column], # stderr
                     label=treatmentset)

    plt.legend(loc=2)

## Analysis

### Basic Sanity Checks

In [ ]:
dataLists_P_1 = read_glob("P_1*/*", "average.dat")
means_P_1,stderrs_P_1 = calc_stats(dataLists_P_1)

In [ ]:
dataLists_P_1

In [ ]:
means_P_1

In [ ]:
plot_column('fitness', means_P_1, stderrs_P_1)

In [ ]:
dataLists_P_0 = read_glob("P_0/*", "average.dat")

#means_P_0,stderrs_P_0 = calc_stats(dataLists_P_0)

In [ ]:
plot_column('fitness', means_P_0, stderrs_P_0)

### Comparing Probability Treatments

In [ ]:
probs = ['1', '0.75', '0.5', '0.25', '0.1', '0.01', '0.001', '0']
datas = {}
means = {}
errors = {}
boostraps = {}
for prob in probs:
    datas[prob] = read_glob("P_"+prob+"/*", "average.dat")
    #print "HI"
    means[prob], errors[prob], bootstraps[prob] = calc_stats(datas[prob])
    

In [ ]:
means.keys()

In [ ]:
means['0'].keys()

In [ ]:
means['0.01']['hgtbonus'].describe()

In [ ]:
means['0.75']['hgtbonus'].describe()

In [ ]:
plot_sets("fitness", "hgtbonus", means, errors)

### Just the mean lines

In [ ]:
def plot_across_treatments(treatmentset, treatment, column):
    #treatmentset = set of probabilities to plot ['1', '0.1'...]
    #treatment = which treatment to plot "hgt" "hgtbonus" etc.
    #column = which column to plot "fitness", "gestation_time" etc.
    data = {}
    means = {}
    errors = {}
    bootstrap_errors = {}
    
    combined_dataframes = pd.DataFrame()
    
    for prob in treatmentset:
        data[prob] = read_glob("P_"+prob+"/*", "average.dat")
        #print "HI"
        means[prob], errors[prob] = calc_stats(data[prob])
        bootstrap_errors[prob] = calc_bootstrap(data[prob], column)
    
    for treatmentset in sorted(means.keys()):
        combined_dataframes[treatmentset] = means[treatmentset][treatment][column]
        
    ax = combined_dataframes.plot(logy=True, 
                              title="fitnesses at various hgtbonus probabilities", 
                              yerr=bootstrap_errors
                             )
    ax.set_xlabel("100x Updates"),
    ax.set_ylabel("log fitness")    


In [ ]:
plot_across_treatments(['1', '0.75', '0.5', '0.25', '0.1', '0.01', '0.001', '0'],
                      "hgtbonus", "fitness")

In [ ]:
#probs = ['1', '0.75', '0.5', '0.25', '0.1', '0.01', '0.001', '0']
probs = ['0.01']
datas_t = {}
means_t = {}
errors_t = {}
bootstraps_t = {}
for prob in probs:
    datas_t[prob] = read_glob("P_"+prob+"/*", "average.dat")
    #print "HI"
    means_t[prob], errors_t[prob] = calc_stats(datas_t[prob])
    bootstraps_t[prob] = calc_bootstrap(datas_t[prob], "fitness")
    
combined_dataframes = pd.DataFrame()
treatment = "hgtbonus"
column = "fitness"
for treatmentset in sorted(means.keys()):
    #if (treatmentset not in ['0.25', '0.5', '0.75']):
    combined_dataframes[treatmentset] = means[treatmentset][treatment][column]
    
ax = combined_dataframes.plot(logy=True, 
                              title="fitnesses at various hgtbonus probabilities", 
                              yerr=combined_dataframes.std()
                             )
ax.set_xlabel("100x Updates"),
ax.set_ylabel("log fitness")

In [ ]:
combined_dataframes.head()

In [ ]:
combined_dataframes.std()

In [ ]:
sorted(combined_dataframes.columns)

In [ ]:
ax = combined_dataframes.plot(logy=True, 
                              title="fitnesses at various hgtbonus probabilities", 
                              yerr=combined_dataframes.std()
                             )
ax.set_xlabel("100x Updates"),
ax.set_ylabel("log fitness")

In [ ]:
dir(combined_dataframes.plot())

In [ ]:
combined_dataframes.describe()

### Now for something new

In [ ]:
from pandas.tools.plotting import bootstrap_plot

In [ ]:
bootstrap_plot(combined_dataframes['0.01'], size=20, samples=500, color='grey')